# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests



# Import API key
from api_keys import geoapify_key

In [2]:
import os
print(os.getcwd())

C:\Users\javie\OneDrive\desktop\Analysis_Projects\python-api-challenge


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,11.93,80,25,16.34,SH,1723655582
1,1,pacific grove,36.6177,-121.9166,19.10,87,20,3.60,US,1723655585
2,2,queenstown,-31.8976,26.8753,8.94,35,0,6.41,ZA,1723655587
3,3,adamstown,-25.0660,-130.1015,20.95,92,88,5.84,PN,1723655588
4,4,hadibu,12.6500,54.0333,26.20,81,41,6.89,YE,1723655590


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color= "City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 28) & 
    (city_data_df["Max Temp"] > 20) & 
    (city_data_df["Wind Speed"] < 6) & 
    (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values
narrowed_df= narrowed_df.dropna()

# Display sample data
narrowed_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,lompoc,34.6391,-120.4579,25.32,75,0,0.45,US,1723655622
31,31,talara,-4.5772,-81.2719,24.42,64,0,5.14,PE,1723655628
48,48,mumbwa,-14.9783,27.0619,22.65,19,0,2.22,ZM,1723655655
138,138,inonu,37.3167,33.7667,27.53,16,0,3.48,TR,1723655794
146,146,colonia,50.9333,6.9500,23.19,85,0,2.57,DE,1723655807


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = narrowed_df[["City","Country","Lng","Lat","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df ["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lng,Lat,Humidity,Hotel Name
27,lompoc,US,-120.4579,34.6391,75,
31,talara,PE,-81.2719,-4.5772,64,
48,mumbwa,ZM,27.0619,-14.9783,19,
138,inonu,TR,33.7667,37.3167,16,
146,colonia,DE,6.9500,50.9333,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "Categories": "Acommodations.hotel",
    "apikey": geoapify_key,
    "Limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = row["Lat"]
    Longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lompoc - nearest hotel: No hotel found
talara - nearest hotel: No hotel found
mumbwa - nearest hotel: No hotel found
inonu - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
runing - nearest hotel: No hotel found
piti village - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
mahanoro - nearest hotel: No hotel found
cafayate - nearest hotel: No hotel found
nagato - nearest hotel: No hotel found
vadso - nearest hotel: No hotel found
zhongxiang - nearest hotel: No hotel found
livingstone - nearest hotel: No hotel found
kapuskasing - nearest hotel: No hotel found
kumamoto - nearest hotel: No hotel found
skelleftea - nearest hotel: No hotel found
kolonia - nearest hotel: No hotel found
alpena - nearest hotel: No hotel found
juan de ayolas - nearest hotel: No hotel found
kuusamo - nearest hotel: No hotel found
piton saint-leu - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
ribeira brava - nearest hotel:

,City,Country,Lng,Lat,Humidity,Hotel Name
27,lompoc,US,-120.4579,34.6391,75,No hotel found
31,talara,PE,-81.2719,-4.5772,64,No hotel found
48,mumbwa,ZM,27.0619,-14.9783,19,No hotel found
138,inonu,TR,33.7667,37.3167,16,No hotel found
146,colonia,DE,6.9500,50.9333,85,No hotel found
168,runing,CN,114.3542,33.0011,94,No hotel found
198,piti village,GU,144.6933,13.4626,89,No hotel found
204,tsiombe,MG,45.4833,-25.3000,74,No hotel found
210,mahanoro,MG,48.8000,-19.9000,78,No hotel found
239,cafayate,AR,-65.9761,-26.0730,6,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"]
    
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)